# Script for Processing Data

In [2]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import seaborn as sns
import matplotlib.pyplot as plt
# from ggplot import *


from ruffus import *
import sys, os, h5py, random, tempfile, scipy, time,copy
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.parse_gct as parse_gct
import pipeline_support as PS
from sklearn.decomposition import PCA
from scipy import stats
from matplotlib.pyplot import imshow
from sklearn.preprocessing import MinMaxScaler

from collections import Counter
#from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow_gan.python.losses import losses_impl


from numpy.random import seed
randomState = 123
seed(randomState)

Parameters

In [3]:
gtex_rnaseq_filename = "../data/GTEx/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct"
gtex_l1000_filename = "../data/GTEx/DS_GTEX_L1000_n3176x12320.gctx"
gtex_geneinfo_filename = "../data/GTEx/GSE92743_Broad_GTEx_gene_info.txt"
gtex_sampleinfo_filename = "../data/GTEx/GSE92743_Broad_GTEx_inst_info.txt"

l1000_all_gene_list = "../data/L1000/all_gene_list.txt"
l1000_landmark_gene_list = "../data/L1000/landmark_gene_list.txt"

archs4_all_gene_list = "../data/ARCHS4/all_gene_list.txt"

gtex_l1000_all_gene_list = "../data/GTEx/l1000_all_gene_list.txt"
gtex_l1000_landmark_gene_list = "../data/GTEx/l1000_landmark_gene_list.txt"

gtex_rnaseq_all_gene_list = "../data/GTEx/rnaseq_all_gene_list.txt"

Output Filenames

In [4]:
gtex_filtered_l1000_output_filename = "../data/processed/GTEx/GSE92743_Broad_GTEx_L1000_Level3_Q2NORM_filtered_n{}x{}_cellline_{}.f" # samplesx962
gtex_filtered_rnaseq_output_filename = "../data/processed/GTEx/GSE92743_Broad_GTEx_RNAseq_Log2RPKM_q2norm_filtered_n{}x{}_cellline_{}.f" # samplesx962



## Load overlap landmark genes

In [5]:
with open(l1000_landmark_gene_list, "r") as f:
    l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(archs4_all_gene_list, "r") as f:
    archs4_all_gene = [x.strip() for x in f.readlines()]
with open(gtex_l1000_landmark_gene_list, "r") as f:
    gtex_l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(gtex_rnaseq_all_gene_list, "r") as f:
    gtex_rnaseq_all_gene = [x.strip() for x in f.readlines()]
    
overlap_landmark_genes = list(set(l1000_landmark_gene).intersection(archs4_all_gene).intersection(gtex_l1000_landmark_gene).intersection(gtex_rnaseq_all_gene))


## Load GTEx 

GTEx L1000 from GSE92742 
GTEx RNA-seq from https://www.gtexportal.org/home/datasets version 8 Gene read count

In [6]:
gtex_gene_info = pd.read_csv(gtex_geneinfo_filename,header = 0, sep = '\t')
gtex_landmark_genes = gtex_gene_info.loc[gtex_gene_info["pr_is_lm"]==1, "pr_gene_symbol"].tolist()

In [6]:
# GTEx L1000 data
print('Loading GTEx L1000 data.....')
gtex_l1000_data = parse_gctx.parse(gtex_l1000_filename,convert_neg_666=True).data_df

# create a probe_id to gene name dictionary 
gtex_gene_dict = dict(zip([str(x) for x in gtex_gene_info['pr_gene_id']], gtex_gene_info['pr_gene_symbol']))

# label rows with gene names 
gtex_l1000_data.index = [gtex_gene_dict[x] for x in gtex_l1000_data.index.values]
gtex_l1000_data = gtex_l1000_data.T

Loading GTEx L1000 data.....


In [20]:
cell_line_tissue_dict = {"MCF7": ["Breast - Mammary Tissue"], 
    "PC3": ["Prostate"], 
    "HA1E": ["Kidney - Cortex"], 
    "HCC515": ["Lung"], 
    "VCAP": ["Prostate"], 
    "A375": ["Skin - Not Sun Exposed (Suprapubic)", "Skin - Sun Exposed (Lower leg)"], 
    "HEPG2": ["Liver"], 
    "HT29": ["Colon - Sigmoid", "Colon - Transverse"], 
    "A549": ["Lung"]
}

In [30]:
sample_info = pd.read_csv(gtex_sampleinfo_filename, sep="\t")
# GTEx RNA-seq data
print('Loading GTEx RNA-seq data.....')

with open(gtex_rnaseq_filename, "r") as f:
    lines = f.readlines()
    
    for cell_line, tissue_name in cell_line_tissue_dict.items():
        print(cell_line, tissue_name)
        # gtex l1000 data for each tissue (cell line)
        sample_ids_tissue = sample_info.loc[sample_info["tissue"].isin(tissue_name), "gtex_id"].tolist()

        gtex_l1000_data_tissue = gtex_l1000_data.loc[sample_ids_tissue, :]

        # filter landmark genes and save
        filtered_gtex_l1000_data = gtex_l1000_data_tissue.loc[:, overlap_landmark_genes]
        
    
        # gtex RNA-seq data for each tissue
        sample_line = lines[2]
        samples = sample_line.split("\t")
        sample_index = [i for i, x in enumerate(samples) if x in filtered_gtex_l1000_data.index]
        paired_sample_id = [x for i, x in enumerate(samples) if x in filtered_gtex_l1000_data.index]

        gex_paired_sample = list()
        gene_names = list()
        for line in lines[3:]:
            splited = np.array(line.split("\t"))
            gene_name = splited[1]

            if gene_name in overlap_landmark_genes:
                gene_names.append(gene_name)
                gex_paired_sample.append(splited[sample_index])

        gtex_rnaseq_data = pd.DataFrame(gex_paired_sample, columns=paired_sample_id, index=gene_names).T
        print(gtex_rnaseq_data.shape)


        # print(filtered_gtex_l1000_data)
        filtered_gtex_l1000_data = filtered_gtex_l1000_data.loc[gtex_rnaseq_data.index, :]
        print(filtered_gtex_l1000_data.shape)

        # save gtex l1000
        filtered_gtex_l1000_data.reset_index().to_feather(gtex_filtered_l1000_output_filename.format(filtered_gtex_l1000_data.shape[0], filtered_gtex_l1000_data.shape[1], cell_line))
        # save gtex RNA-seq
        gtex_rnaseq_data.reset_index().to_feather(gtex_filtered_rnaseq_output_filename.format(gtex_rnaseq_data.shape[0], gtex_rnaseq_data.shape[1], cell_line))

Loading GTEx RNA-seq data.....
MCF7 ['Breast - Mammary Tissue']
(43, 962)
(43, 962)
PC3 ['Prostate']
(41, 962)
(41, 962)
HA1E ['Kidney - Cortex']
(5, 962)
(5, 962)
HCC515 ['Lung']
(133, 962)
(133, 962)
VCAP ['Prostate']
(41, 962)
(41, 962)
A375 ['Skin - Not Sun Exposed (Suprapubic)', 'Skin - Sun Exposed (Lower leg)']
(151, 962)
(151, 962)
HEPG2 ['Liver']
(38, 962)
(38, 962)
HT29 ['Colon - Sigmoid', 'Colon - Transverse']
(103, 962)
(103, 962)
A549 ['Lung']
(133, 962)
(133, 962)


In [7]:
sample_info = pd.read_csv(gtex_sampleinfo_filename, sep="\t")

In [15]:
sample_info.groupby("tissue").count().sort_values("gtex_id", ascending=False).to_csv("GTEx_counts_by_tissue.csv")